# Oppgave 4
Her skal du implementere et Pythonprogram (med bruk av SQL) som tar inn
en dato og skriver ut hvilke forestillinger som finnes på denne datoen og lister
opp hvor mange billetter (dvs. stoler) som er solgt. Ta også med forestillinger
hvor det ikke er solgt noen billetter.

In [6]:
import sqlite3
import pandas as pd
from tabulate import tabulate
from utils import get_file_name_of_newest_database

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

In [7]:
def hent_forestilling_med_dato(date):
    # get forestilling with date and number of tickets sold
    query = """
    SELECT 
    F.TeaterStykkeNavn,
    F.Nummer as ForestillingNummer,
    F.Dato,
    F.Tid,
    count(B.ForestillingNummer) as AntallBilletter
    FROM Forestilling F
    LEFT JOIN Billett B
    ON F.Nummer = B.ForestillingNummer
    AND F.TeaterStykkeNavn = B.TeaterStykkeNavn
    Group BY F.TeaterStykkeNavn, F.Nummer
    HAVING F.Dato = ?
    """
    
    cursor.execute(query, (date,))
    # cursor.execute(query)
    forestillinger = cursor.fetchall()
    headers = [description[0] for description in cursor.description]
    return forestillinger, headers


# for forestilling in hent_forestilling_med_dato("2024-02-03"):
# print(f'{forestilling[1]} går den {forestilling[2]} kl {forestilling[3]}')

# forestillings_dato = input("Oppagve 4: Skriv inn datoen for forestillingen (YYYY-MM-DD): ")
# forestillings_dato = "2024-02-03"
forestillings_dato = input('Skriv en dato på dette formatet: ÅÅÅÅ-MM-DD')
forestilling_data, headers = hent_forestilling_med_dato(forestillings_dato)
print(tabulate(forestilling_data, headers=headers , tablefmt="grid"))

+--------------------+----------------------+--------+-------+-------------------+
| TeaterStykkeNavn   | ForestillingNummer   | Dato   | Tid   | AntallBilletter   |
+====================+======================+========+=======+===================+
+--------------------+----------------------+--------+-------+-------------------+
